In [2]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV

In [3]:
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score, accuracy_score

In [4]:
# считываем данные
df = pd.read_csv('article_data.csv', sep = '\t')
df.head(10)

,Article,Len,BadFreqs,Len1,Len2,Len3,Len4,Class
0,#перенаправление Баскервилла,2,0,0,0,0,0,bad
1,'''Бифренария''' ( ) – Орхидной будосъёс (Orc...,81,7,0,1,1,0,bad
2,#перенаправление Бифренария,2,0,0,0,0,0,bad
3,'''Бинотия''' ( ) – Орхидной будосъёс (Orchid...,35,5,0,1,0,0,bad
4,#перенаправление Бинотия,2,0,0,0,0,0,bad
5,'''Бипиннула''' ( ) – Орхидной будосъёс (Orch...,131,5,0,1,0,0,bad
6,#перенаправление Бипиннула,2,0,0,0,0,0,bad
7,'''Блетия''' ( ) – Орхидной будосъёс (Orchida...,173,14,0,6,0,6,bad
8,#перенаправление Блетия,2,0,0,0,0,0,bad
9,'''Блетилла''' ( ) – Орхидной будосъёс (Orchi...,49,5,0,1,0,0,bad


In [5]:
# проверяем баланс классов
print(('Доля объектов класса bad в обучающей выборке'))
print((len(df[df.Class == 'bad']))/df.Class.count())
# print((len(df[df.Class == 'good']))/df.Class.count())

Доля объектов класса bad в обучающей выборке
0.858890469417


In [6]:
# признаки и целевая переменная
y = df.Class
X = np.column_stack((np.array(df.Len), np.array(df.BadFreqs), np.array(df.Len2), np.array(df.Len3), np.array(df.Len4)))
print (X)

[[ 2  0  0  0  0]
 [81  7  1  1  0]
 [ 2  0  0  0  0]
 ..., 
 [ 3  0  1  0  0]
 [47  0  0  0  0]
 [ 2  0  0  0  0]]


In [7]:
# перемешиваем и делим выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# обучаем решающий лес, параметр n_estimators подбирается с помощью класса GridSearchCV
param_grid = {'n_estimators' : np.arange(10, 100)}
frst = GridSearchCV(RandomForestClassifier(), param_grid)
frst.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': array([10, 11, ..., 98, 99])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [58]:
# проверка обученного классификатора на тестовой выборке
y_pred = frst.predict(X_test)
target_names = ['bad', 'good']
print(classification_report(y_test, y_pred, target_names=target_names)) 

             precision    recall  f1-score   support

        bad       0.98      0.96      0.97       605
       good       0.78      0.86      0.82        98

avg / total       0.95      0.95      0.95       703



In [10]:
print('Доля верных ответов классификатора:')
print(frst.score(X_test, y_test))
print('Доля плохих статей, определенных как хорошие:')
print(1 - (recall_score(y_test, y_pred, pos_label = 'bad')))
print('Доля хороших статей, определенных как плохие:')
print(1 - (recall_score(y_test, y_pred, pos_label = 'good')))

Доля верных ответов классификатора:
0.945945945946
Доля плохих статей, определенных как хорошие:
0.0396694214876
Доля хороших статей, определенных как плохие:
0.142857142857


In [11]:
# сравнение с baseline-классификатором по длине
# все статьи короче 30 слов определяются как плохие
y_pred_base = []
for i in X_test[:, 0]:
    if i >= 30:
        y_pred_base.append('good')
    else:
        y_pred_base.append('bad')
print('Доля верных ответов baseline-классификатора:')
print (accuracy_score(y_test, y_pred_base))
print('Доля плохих статей, определенных как хорошие у baseline-классификатора:')
print(1 - (recall_score(y_test, y_pred_base, pos_label = 'bad')))
print('Доля хороших статей, определенных как плохие у baseline-классификатора:')
print(1 - (recall_score(y_test, y_pred_base, pos_label = 'good')))

Доля верных ответов baseline-классификатора:
0.778093883357
Доля плохих статей, определенных как хорошие у baseline-классификатора:
0.226446280992
Доля хороших статей, определенных как плохие у baseline-классификатора:
0.19387755102


In [12]:
# чтобы проверить важность признаков, посмотрим на feature_importances_, чем больше этот параметр, тем важнее признак
feature_names = ['Len', 'BadFreqs', 'Len2', 'Len3', 'Len4']
for i in range(len(feature_names)):
    print (feature_names[i]+':  '+str(frst.best_estimator_.feature_importances_[i]))

Len:  0.269830862195
BadFreqs:  0.153682524756
Len2:  0.196394899069
Len3:  0.133767725605
Len4:  0.246323988374


In [118]:
# тест на данных мокшанского языка
# проверяем баланс классов
df2 = pd.read_csv('mdfwiki_data.csv', sep = '\t')
print(('Доля объектов класса bad в тестовой выборке'))
print((len(df2[df2.Class == 'bad']))/df2.Class.count())
y_test2 = np.array(df2.Class)
X_test2 = np.column_stack((np.array(df2.Len), np.array(df2.BadFreqs), np.array(df2.Len2), np.array(df2.Len3), np.array(df2.Len4)))
print (len(y_test2))

Доля объектов класса bad в тестовой выборке
0.678125
320


In [127]:
y_pred2 = frst.predict(X_test2)
target_names = ['bad', 'good']
print(classification_report(y_test2, y_pred2, target_names=target_names)) 

             precision    recall  f1-score   support

        bad       0.87      0.97      0.92       217
       good       0.92      0.70      0.80       103

avg / total       0.89      0.88      0.88       320



In [120]:
print('Доля верных ответов классификатора для мокшанского языка:')
print(frst.score(X_test2, y_test2))
print('Доля плохих статей, определенных как хорошие:')
print(1 - (recall_score(y_test2, y_pred2, pos_label = 'bad')))
print('Доля хороших статей, определенных как плохие:')
print(1 - (recall_score(y_test2, y_pred2, pos_label = 'good')))

Доля верных ответов классификатора для мокшанского языка:
0.884375
Доля плохих статей, определенных как хорошие:
0.0276497695853
Доля хороших статей, определенных как плохие:
0.300970873786


In [124]:
# тест на данных башкирского языка
# проверяем баланс классов
df3 = pd.read_csv('bawiki_data.csv', sep = '\t')
print(('Доля объектов класса bad в тестовой выборке'))
print((len(df3[df3.Class == 'bad']))/df3.Class.count())
y_test3 = np.array(df3.Class)
X_test3 = np.column_stack((np.array(df3.Len), np.array(df3.BadFreqs), np.array(df3.Len2), np.array(df3.Len3), np.array(df3.Len4)))
print (len(y_test3))

Доля объектов класса bad в тестовой выборке
0.675
320


In [128]:
y_pred3 = frst.predict(X_test3)
target_names = ['bad', 'good']
print(classification_report(y_test3, y_pred3, target_names=target_names)) 

             precision    recall  f1-score   support

        bad       0.81      0.83      0.82       216
       good       0.64      0.61      0.62       104

avg / total       0.76      0.76      0.76       320



In [126]:
print('Доля верных ответов классификатора для башкирского языка:')
print(frst.score(X_test3, y_test3))
print('Доля плохих статей, определенных как хорошие:')
print(1 - (recall_score(y_test3, y_pred3, pos_label = 'bad')))
print('Доля хороших статей, определенных как плохие:')
print(1 - (recall_score(y_test3, y_pred3, pos_label = 'good')))

Доля верных ответов классификатора для башкирского языка:
0.759375
Доля плохих статей, определенных как хорошие:
0.166666666667
Доля хороших статей, определенных как плохие:
0.394230769231
